# Tool Call Without Pydantic 

In [ ]:
# Installs for the below cells.

%pip install openai
%pip install langchain
%pip install langchain-openai

In [ ]:
import os

# # Set OPENAI API Key

os.environ["OPENAI_API_KEY"] = "your openai key"

# OR (load from .env file)
# make sure you have python-dotenv installed
# from dotenv import load_dotenv
# load_dotenv("./.env")

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.tools import tool

In [3]:
@tool
def find_name_info_from_database(name):
    """REturns information about a name from a database."""
    names_info = {
        'Alice': {'Age': 25, 'Gender': 'F'},
        'Bob': {'Age': 30, 'Gender': 'M'},
        'Charlie': {'Age': 35, 'Gender': 'M'}
    }
    
    return names_info[name]

find_name_info_from_database.invoke("Alice")

{'Age': 25, 'Gender': 'F'}

In [4]:
tools = [find_name_info_from_database]
llm = ChatOpenAI(model="gpt-4o")

llm_with_tools = llm.bind_tools(tools)

In [5]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Output(BaseModel):
    info : dict = Field(..., description="Information about the person")
    

prompt = ChatPromptTemplate.from_template("You take in a request about a person and call a function to return information about that person: {name}")

chain = prompt | llm_with_tools

tool_calls = chain.invoke({"name": "Alice"})

tool_calls

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Cxxg6M49lBGNJc2D3oubRI0D', 'function': {'arguments': '{"name":"Alice"}', 'name': 'find_name_info_from_database'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 71, 'total_tokens': 88}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-43fd4d71-b6b2-4c55-8d57-8b4e107c60d8-0', tool_calls=[{'name': 'find_name_info_from_database', 'args': {'name': 'Alice'}, 'id': 'call_Cxxg6M49lBGNJc2D3oubRI0D'}], usage_metadata={'input_tokens': 71, 'output_tokens': 17, 'total_tokens': 88})

In [6]:
output = tool_calls.tool_calls[0]
output

{'name': 'find_name_info_from_database',
 'args': {'name': 'Alice'},
 'id': 'call_Cxxg6M49lBGNJc2D3oubRI0D'}

In [7]:
selected_tool = {"find_name_info_from_database": find_name_info_from_database}[output["name"].lower()]
selected_tool

StructuredTool(name='find_name_info_from_database', description='REturns information about a name from a database.', args_schema=<class 'pydantic.v1.main.find_name_info_from_databaseSchema'>, func=<function find_name_info_from_database at 0x10bb11d00>)

In [8]:
type(selected_tool)

langchain_core.tools.StructuredTool

In [9]:
tool_output = selected_tool.invoke(output["args"])
tool_output

{'Age': 25, 'Gender': 'F'}

# Tool Call With Pydantic

In [10]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.pydantic_v1 import BaseModel, Field

class NameInfo(BaseModel):
    age: int = Field(..., description="Age of the person")
    status: str = Field(..., description="Gender of the person")
                      
prompt = ChatPromptTemplate.from_template("You take in a request and return info about these people: \
                                           Bob: [34, married], Mary: [25, single], Alice: [30, single],\
                                          Request: {name}.")
llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools([NameInfo])

llm_with_tools

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10bd56c90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10bd3f2d0>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_api_base='https://api.openai.com/v1', openai_proxy=''), kwargs={'tools': [{'type': 'function', 'function': {'name': 'NameInfo', 'description': '', 'parameters': {'type': 'object', 'properties': {'age': {'description': 'Age of the person', 'type': 'integer'}, 'status': {'description': 'Gender of the person', 'type': 'string'}}, 'required': ['age', 'status']}}}]})

In [12]:
chain = prompt | llm_with_tools | PydanticToolsParser(tools=[NameInfo])

output_info = chain.invoke({"name": "Alice"})

In [15]:
output_info[0].age

30

In [16]:
output_info[0].status

'single'

In [ ]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.messages import HumanMessage

model = OllamaFunctions(
    model="llama3", 
    format="json"
    )

model = model.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)

response = model.invoke("what is the weather in Singapore?")

print(response)